In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch 
import torch.nn.functional as F #Functionals like parameters like acticaition functions
import torchvision.datasets as datasets #standard dataset
import torchvision.transforms as transforms #transformations we can perform on our datasets: like data agumentation
from torch import optim  #For optimizers sigmoid, adam etc
from torch import nn #all neural network modules
from torch.utils.data import DataLoader  #Gives easier dataset management by creating mini batches etc.
from tqdm import tqdm #For a nice progress bar!

In [3]:
# set device
device= "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
#Hyperparameters

input_size= 28
hidden_size= 256
num_layers= 2
num_classes= 10
sequence_length= 28
learning_rate= 0.005
batch_size= 64
num_epochs= 3

In [5]:
#LTSM Model

class RNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(
            x, (h0, c0)
        )  # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out

In [6]:
#Load Data

train_dataset= datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download= True)

test_dataset= datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download= True)

train_loader= DataLoader(dataset= train_dataset, batch_size=batch_size, shuffle= True)
test_loader= DataLoader(dataset= test_dataset, batch_size= batch_size, shuffle= True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 48713906.04it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1457446.84it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 11480792.11it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1991899.70it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [7]:
model= RNN_LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

In [8]:
#Loss and optimizer

criterion= nn.CrossEntropyLoss()
optimizer= optim.Adam(model.parameters(), lr= learning_rate)

In [9]:
#Train network

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        #Get data to cuda or cpu
        data= data.to(device=device).squeeze(1)
        targets= targets.to(device=device)
        
        #Forward
        scores= model(data)
        loss= criterion(scores, targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #gradient descent update step/adam step
        optimizer.step()

100%|██████████| 938/938 [02:14<00:00,  6.97it/s]


In [10]:
#Check accuracy on training & Test to see how good our model

def check_accuracy(loader, model):
    num_correct= 0
    num_samples= 0
    
    #set model to eval
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x= x.to(device=device).squeeze(1)
            y= y.to(device=device)
            
            scores= model(x)
            _, predictions= scores.max(1)
            num_correct +=(predictions==y).sum()
            num_samples += predictions.size(0)
        
        
    model.train()
    return num_correct/ num_samples

In [11]:
print(f" Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
print(f" Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

 Accuracy on training set: 99.048332
 Accuracy on test set: 98.68
